# ДЗ 3

In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter
from operator import itemgetter

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def ngrammer(tokens, n):
    ngrams = []
    tokens = [token for token in tokens]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

In [2]:
# чуть-чуть ограничил, т.к. сичтается онь долго
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()[:100]
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()[:100]

In [3]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [4]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()[:100000]

In [5]:
WORDS = Counter(re.findall('\w+', corpus.lower()))

In [6]:
# фунцкия расчета вероятности слова
N = sum(WORDS.values())
def P(word, N=N): 
    "Вычисляем вероятность слова"
    return WORDS[word] / N

Чтобы найти исправления нужно сгенерировать возможные исправления и выбрать те, которые есть в словаре. Если есть несколько вариантов, то выбрать тот, у котогоро наибольшая вероятность.

In [7]:
def correction(word): 
    candidate = word # если не наши подходящего слова в сгенерированных
    all_del = list(edits1(word).union(set([t for t in edits2(word)]))) # генерируем все удаления для данного слова
    for word_del in all_del: # обходим всем варианты удаления
        for key in WORDS_with_deletions: # проверяем во всех вариантах словаря
            if word_del in WORDS_with_deletions[key]:
                candidate = key # если нашли, то сразу определяем слово, т.к. оно наиболее вероятное
                return candidate
    return candidate

def edits1(word):
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return set(deletes)

def edits2(word): 
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [8]:
# сортируем dict по значению чтобы при обходе сразу выбиралось амое частотное слово
tmp_dict = dict(WORDS)
WORDS_sort = dict(sorted(tmp_dict.items(), key=itemgetter(1), reverse=True)) 

In [12]:
WORDS_with_deletions = {}
for word in WORDS_sort:
    WORDS_with_deletions[word] = edits1(word).union(set([t for t in edits2(word)]))

Для оценки используем будем использовать три метрики:  
1) процент правильных слов;  
2) процент исправленных ошибок  
3) процент ошибочно исправленных правильных слов

In [13]:
%%time
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)
        

0
CPU times: user 16.6 s, sys: 0 ns, total: 16.6 s
Wall time: 16.8 s


In [14]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.3465346534653465
0.11724137931034483
0.6190476190476191


работает быстрее, но точность все также невелика